In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager

service = Service("C:/Users/USER/Desktop/chromedriver.exe")
driver = webdriver.Chrome(service=service)  # 수정된 부분
driver.implicitly_wait(2)  # 웹페이지 로딩 대기 시간 설정

########################################## 사용자 설정 ############################################

keywords = ['프리미어리그','프리메라리가','분데스리가','리그앙','세리에A','챔피언스리그','FA컵','월드컴','카라바오컵'] # 검색 키워드
sort = 1 # 0: 관련도순 1 : 최신순
search_page = 4 # 수집제한 페이지

##################################################################################################

search_num = 1 # 수집한 기사수 확인용 변수설정
data = [] # 리스트형 변수 설정

for keyword in keywords:
    for i in range(search_page) :
    
        # 페이지별 start_num 추출
        if i == 0 :
            start_num = 1
        else :
            start_num = (i * 10) + 1
    
        url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&sort={sort}&start={start_num}'
    
        driver.get(url)
        time.sleep(3)
    
        # BS4 활용시 필요한 코드
        req = driver.page_source 
        soup = BeautifulSoup(req, 'html.parser')
    
    
        # 뉴스 섹션 갖고오기
        # articles = driver.find_elements(By.CSS_SELECTOR, 'ul.list_news > li') # 셀레니움 버전
        articles = soup.select('ul.list_news > li') # BS4 버전
    
        for article in articles :
            
            # media_name = article.find_element(By.CSS_SELECTOR, '.info.press').text # 언론사명 가져오기(셀레니움 버전)
            # title = article.find_element(By.CSS_SELECTOR, '.news_tit').text # 제목 가져오기(셀레니움 버전)
            # content = article.find_element(By.CSS_SELECTOR, '.dsc_wrap').text # 요약내용 가져오기(셀레니움 버전)
    
            media_name = article.select_one('.info.press').text # 언론사명 가져오기(BS4 버전)
            title = article.select_one('.news_tit').text # 제목 가져오기(BS4 버전)
            content = article.select_one('.dsc_wrap').text # 요약내용 가져오기(BS4 버전)
    
    
            print('------------------------------------------------------------')
            print(f'{search_num}, [{media_name}], {title}')
            print(content)
    
            # 데이터를 리스트에 추가
            data.append({
                'Media': media_name,
                'Title': title,
                'Content': content
            })
    
            search_num += 1

df = pd.DataFrame(data)
with open("data.txt", "w", encoding="utf-8") as f:
    f.write(df.to_string(index=False))  

------------------------------------------------------------
1, [스포티비뉴스], 김혜성 마이너리그행이라고? 오히려 주전 2루수 예상! MLB.com "주전 기회... 
 메이저리그 경력에서 외야수로 가장 많이 나섰으나 그 다음으로는 2루수와 유격수를 가장 많이 봤다. 김혜성의 영역과 상당 부분 겹친다. 1년 계약이라는 점에서 다저스도 에르난데스를 장기적인 카드로 보는 건 아니다. 하지만 김혜성은 계약에서 마이너리그 거부권이 없다. 김혜성이 스프링트레이닝에서 못...  
------------------------------------------------------------
2, [마니아타임즈], 팬그래프 예측 이정후 2025시즌 '타율 .293·14홈런'...팬들 '과대평가' ... 
 이정후는 지난해 6년 1억 1300만 달러의 대형 계약으로 메이저리그에 진출했다. 시즌 초반 타율 .316, OPS .849로 순조로운 출발을 보였으나, 5월 중순 어깨 부상으로 37경기 출전에 그쳤다. 최종 성적은 타율 .262, 2홈런 8타점, WAR 0.1을 기록했다. 샌프란시스코 구단은 이정후가 어깨 수술 후 재활을...  
------------------------------------------------------------
3, [일간스포츠], 다저스, 16일부터 스프링캠프 시작...MLB닷컴 "김혜성 눈여겨볼 새 얼굴... 
 메이저리그(MLB) 공식 홈페이지 MLB닷컴 다저스 페이지는 '스프링캠프의 모든 것'이라는 글로 다저스팬들의 시선을 모았다. MBL닷컴은 오는 16일(한국시간) 투수조·포수조·야수조가 모두 모여 미국 애리조나주 글렌데일 캐멀백랜치에서 다저스 스프링캠프가 시작한다고 알렸고, 팬들의 훈련 관람 여부와...  
------------------------------------------------------------
4, [스포티비뉴스], 주전 2루수 유력→마이너행 가능성…'키케 재계약